In [1]:
import os
from keras.models import Sequential,load_model,Model
from keras.applications import InceptionV3
from keras.layers import MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Flatten, Dense, Dropout, Conv2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
data_dir = "traindata/"
classes = os.listdir(data_dir)
classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry_(including_sour)___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Citrus_Black spot',
 'Citrus_canker',
 'Citrus_greening',
 'Citrus_healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Rice_Bacterial_leaf_blight',
 'Rice_Brown_spot',
 'Rice_healthy',
 'Rice_hispa',
 'Rice_LeafBlast',
 'Rice_Leaf_Smut',
 'Rice_sheath_blight',
 'Rice_tungro',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',

In [3]:
import numpy as np
import pandas as pd

image_counts={}
for c in classes:
    try:
        path = data_dir + c
        count = len(os.listdir(path))
        image_counts[c]= count
    except:
        pass

df = pd.DataFrame.from_dict(image_counts, orient='index')
df

,0
Apple___Apple_scab,484
Apple___Black_rot,477
Apple___Cedar_apple_rust,420
Apple___healthy,482
Cherry_(including_sour)___healthy,436
Cherry_(including_sour)___Powdery_mildew,401
Citrus_Black spot,151
Citrus_canker,143
Citrus_greening,184
Citrus_healthy,38


In [4]:
model = Sequential()
#Layer 1
#Conv Layer 1
model.add(Conv2D(filters = 6, 
                 kernel_size = 5, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (32,32,3)))


model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Layer 2
#Conv Layer 2
model.add(Conv2D(filters = 16, 
                 kernel_size = 5,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Flatten
model.add(Flatten())
#Layer 3
#Fully connected layer 1
model.add(Dense(units = 120, activation = 'relu'))
#Layer 4
#Fully connected layer 2
model.add(Dense(units = 84, activation = 'relu'))
#Layer 5
#Output Layer
model.add(Dense(units = 45, activation = 'softmax'))

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size =64

In [8]:
train_generator = train_datagen.flow_from_directory('./traindata/',
                                                 target_size=(32, 32),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory('./validation/',
                                            target_size=(32, 32),
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 22625 images belonging to 45 classes.
Found 917 images belonging to 45 classes.


In [9]:
class_dict = train_generator.class_indices
print(class_dict)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Cherry_(including_sour)___Powdery_mildew': 4, 'Cherry_(including_sour)___healthy': 5, 'Citrus_Black spot': 6, 'Citrus_canker': 7, 'Citrus_greening': 8, 'Citrus_healthy': 9, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 10, 'Corn_(maize)___Common_rust_': 11, 'Corn_(maize)___Northern_Leaf_Blight': 12, 'Corn_(maize)___healthy': 13, 'Grape___Black_rot': 14, 'Grape___Esca_(Black_Measles)': 15, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 16, 'Grape___healthy': 17, 'Peach___Bacterial_spot': 18, 'Peach___healthy': 19, 'Pepper__bell___Bacterial_spot': 20, 'Pepper__bell___healthy': 21, 'Potato___Early_blight': 22, 'Potato___Late_blight': 23, 'Potato___healthy': 24, 'Rice_Bacterial_leaf_blight': 25, 'Rice_Brown_spot': 26, 'Rice_LeafBlast': 27, 'Rice_Leaf_Smut': 28, 'Rice_healthy': 29, 'Rice_hispa': 30, 'Rice_sheath_blight': 31, 'Rice_tungro': 32, 'Strawberry___Leaf_scorch': 33, 'Straw